#### IMPORTS

In [3]:
#Importa os módulos usados

import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
from typing import Dict, List
import copy


# Define um tipo de dado similar ao Pascal "record" or C "struct"

class Struct:
    pass

INVALID: int = -1

#### Utils de leitura do CSV

In [9]:
class Cliente:
    def __init__(self, x, y, consumo_banda):
        self.x = x
        self.y = y
        self.consumo_banda = consumo_banda
        self.connected_PA_id = INVALID
        self.id = INVALID

    def __repr__(self):
        return f"Cliente(x={self.x}, y={self.y}, consumo_banda={self.consumo_banda})"

# Função para ler os clientes do arquivo
def ler_clientes(nome_arquivo):
    clientes = []
    with open(nome_arquivo, 'r') as arquivo:
        linhas = arquivo.readlines()
        for linha in linhas:
            x, y, consumo_banda = map(float, linha.strip().split(','))
            cliente = Cliente(x, y, consumo_banda)
            clientes.append(cliente)

        index = 0
        for cliente in clientes:
            cliente.id = index
            index += 1

    return clientes

In [10]:
def clientProcessing(clientList, nCL=495, nPA=81**2):

    cons = np.zeros(nCL)
    dist = np.zeros((nCL, nPA))
    exp = np.zeros((nCL, nPA))

    cl_index = 0
    for cliente in clientList:
        cons[cl_index] = cliente.consumo_banda

        for PA_index in range(nPA):
            PA_x = (PA_index % 81) * 5
            PA_y = (PA_index // 81) * 5
            
            dist[cl_index, PA_index] = np.sqrt((cliente.x - PA_x)**2 + (cliente.y - PA_y)**2)
            exp[cl_index, PA_index] = 1/dist[cl_index, PA_index]

        cl_index += 1

    return cons, dist, exp

#### Classe Solução

In [15]:
class Solucao:
    id: int = 0
    sol_PAs_ativos: Dict[int, int] = {}
    sol_PA_id_por_cliente: Dict[int, int] = {}
    fitness_PA_min: float = 0
    fitness_PA_min_penalizado: float = 0
    fitness_dist_min: float = 0
    fitness_dist_min_penalizado: float = 0
    porcentagem_CL_antendidos = 0
    nCL_por_PA: Dict[int, int] = {}

    def __init__(self) -> None:
        self.id: int = 0
        self.sol_PAs_ativos: Dict[int, int] = {}
        self.sol_PA_id_por_cliente: Dict[int, int] = {}
        self.fitness_PA_min: float = 0
        self.fitness_PA_min_penalizado: float = 0
        self.fitness_dist_min: float = 0
        self.fitness_dist_min_penalizado: float = 0
        self.porcentagem_CL_antendidos = 0
        self.nCL_por_PA: Dict[int, int] = {}

#### Dados do problema

In [6]:
class AccessPoint:
    id: int = 0
    x: int = 0
    y: int = 0
    nCL_in_range: int = 0

In [16]:
class ProblemData:
    PA_list: List[AccessPoint] = []
    CL_list: List[Cliente] = []
    dist_PA_CL: Dict[int, Dict[int, float]] = np.empty((81**2, 495))
    min_CL_rate: float = 0.05
    nCL: int = 495
    nPA: int = 81**2

In [13]:
def probdef(nPA=81**2, nCL=495):
    
    probdata = ProblemData()

    # Uso da função para ler os clientes do arquivo
    probdata.CL_list = ler_clientes('data/clientes.csv')

    for id in range(0, nPA):
        PA_x = (id % 81) * 5
        PA_y = (id // 81) * 5
        probdata.PA_list.append(PontoDeAcesso(PA_x, PA_y, id))

    for PA in probdata.PA_list:
        for CL in probdata.CL_list:
            probdata.dist_PA_CL[PA.id][CL.id] = ((PA.x - CL.x)**2 + (PA.y - CL.y)**2) ** 0.5

            if(probdata.dist_PA_CL[PA.id][CL.id] <= 84):
                PA.nCL_in_range += 1
    
    return probdata

    probdata.PA_cap = 54 # em Mbps
    probdata.PA_raio = 84 # em metros
    probdata.CL_min_p = 0.05 # porcentagem minima de clientes
    probdata.nPA_max = 30

    return probdata

#### Solução inicial

In [14]:
# heuristica inserção mais proxima
# Solução inicial
def sol_inicial(probdata):
    sol = Solucao()
    
     # Calcular distância euclidiana entre clientes e pontos de acesso
    distancias = np.zeros((probdata.nCL, probdata.nPA))
    for i, CL in enumerate(probdata.CL_list):
        for j, PA in enumerate(probdata.PA_list):
            distancias[i, j] = np.sqrt((CL.x - PA.x) ** 2 + (CL.y - PA.y) ** 2)

    # Atribuir cada cliente ao ponto de acesso mais próximo
    for j in range(len(clientes)):
        distancias_cliente = distancias[j]
        pa_mais_proximo = np.argmin(distancias_cliente)
        sol.sol_PA_id_por_cliente[j] = pa_mais_proximo  # Dá o PA mais proximo como ativo para o cliente j
        sol.sol_PAs_ativos[pa_mais_proximo] = 1  # Ativa o PA mais próximo

    return sol

#### Função Penalidades Generalizadas

In [17]:
# Função de penalidades para todas as restrições
def penalties(sol, probdata):

    # percentual mínimo de clientes
    pen_CLmin = probdata.nCL * probdata.CL_min_p - np.sum(sol.sol_PAs_ativos.values())
    pen_CLmin = np.sum(np.where(pen_CLmin <= 0, 0, pen_CLmin)**2)

    # limite de consumo dos PAs
    pen_PAcap = np.zeros(probdata.nPA)
    for i in range(probdata.nPA):
        #Acha a lista de id de clientes que possuem conexão ativa com o PA i
        CLs_ids = sol_PA_id_por_cliente.keys()[sol_PA_id_por_cliente.values.index(i)]

        consumo_total = 0
        index = 0
        for CL in (probdata.CL_list):
            if index in CLs_ids:
                consumo_total += CL.consumo_banda
            index += 1
        
        current_PA_ativo = 0
        if i in sol.sol_PAs_ativos:
            current_PA_ativo = 1

        pen_PAcap[i] = consumo_total - current_PA_ativo * probdata.PA_cap
        pen_PAcap[i] = np.sum(np.where(pen_PAcap[i] <= 0, 0, pen_PAcap[i])**2)
    
    # limite de distância entre PAs e clientes
    pen_dist = np.zeros((probdata.nCL, probdata.nPA))
    for i, PA in enumerate(probdata.PA_list):
        #Acha a lista de id de clientes que possuem conexão ativa com o PA i
        CLs_ids = sol_PA_id_por_cliente.keys()[sol_PA_id_por_cliente.values.index(i)]
        for j, CL in enumerate(probdata.CL_list):

            areTheyConnected = 0
            if CL.id in CLs_ids:
                areTheyConnected = 1
            
            current_PA_ativo = 0
            if PA.id in sol.sol_PAs_ativos:
                current_PA_ativo = 1

            pen_dist[j,i] = probdata.dist_PA_CL[PA.id, CL.id] * areTheyConnected - current_PA_ativo * probdata.PA_raio
            pen_dist[j,i] = np.sum(np.where(pen_dist[j,i] <= 0, 0, pen_dist[j,i])**2)

    # pelo menos 5% de exposição à rede
    pen_CLmin = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_CLmin[j] = 0.05 * 1 - np.dot(np.array(probdata.exp_CL_PA[j]), np.array(y.solution))
        pen_CLmin[j] = np.sum(np.where(pen_CLmin[j] <= 0, 0, pen_CLmin[j])**2)
    
    # no máximo um PA por cliente
    pen_PAperCL = np.zeros(probdata.nCL)
    for j in range(probdata.nCL):
        pen_PAperCL[j] = sum(x.solution[j]) - 1
        pen_PAperCL[j] = np.sum(np.where(pen_PAperCL[j] <= 0, 0, pen_PAperCL[j])**2)

    # número máximo de PAs
    pen_PAmax = sum(sol.sol_PAs_ativos.values()) - probdata.nPA_max

    # return all multiplied by U
    return 1000 * (pen_CLmin + sum(pen_PAcap) + sum(pen_dist) + sum(pen_CLmin) + sum(pen_PAperCL) + pen_PAmax)

#### Funçao Objetivo 1: Minimizar número de PAs ativos

#### Função Objetivo 2: Minimizar distâncias entre clientes e PAs